# Топографическая коррекция

В данном документе производится топографическая коррекция сцен Ландсат.

Для того, чтобы произвести топографическую коррекцию нам необходимы данные по высотам рельефа. Следовательно, нам потребуется импортировать высотные данные. В качестве таких данных возьмем SRTM. (Есть ли на Дальний Восток данные получше?). Таким образом схема работы будет такой:

1. Импорт SRTM в набор данных GRASS.
2. Коррекция всех имеющихся сцен Landsat.


## Импорт STRM

Поскольку данные STRM растпространяются в системе координат широта-долгота, то сначала импортируем данные в область, соответствующую системе координат исходных данных, а затем перепроециуем их в нужную нам проекцию.

In [9]:
import os

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

In [11]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_ll_location_name()
)

grs.grass.run_command('g.mapset', mapset='srtm', flags='c')

0

Импортируем данные по высотам:

In [12]:
HOME_DIR = os.getenv("HOME")
ELEV_DIR = os.path.join(HOME_DIR, 'Data', 'SRTM')

print ELEV_DIR

/home/cruncher/Data/SRTM


In [14]:
basenames = ['srtm_63_03', 'srtm_64_03']
for bs in basenames:
    name = os.path.join(ELEV_DIR, bs+'.tif')
    print bs
    grs.grass.run_command('r.in.gdal', input=name, output=bs, overwrite=True, flags='e')
    

srtm_63_03
srtm_64_03


Объединим разные куски SRTM в один для удобства работы, а отдельные части удалим:

In [18]:
grs.grass.run_command(
    'r.patch', input=','.join(basenames), 
    output='srtm',
    overwrite=True
)
for bs in basenames:
    grs.grass.run_command('g.remove', type='rast', name=bs, flags='f')

Перепроецируем, для этого переключимся в область UTM и создадим отдельный набор данных для хранения высот:

In [19]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='elevation', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

elevation



Установим регион, покрывающий все интересующие нас сцены:

In [22]:
grs.grass.run_command('g.region', region='all_scenes@landsat')

0

Перепроецируем:

In [23]:
grs.grass.run_command('r.proj', location=get_ll_location_name(), mapset='srtm', input='srtm', output='srtm')

0

## Топографическая коррекция

Для топографической коррекци нам нужно знать параметры освещенности в момент съемки: азимут на Солнце и зенитный угол. Эти параметры предоставляются в метаданных к снимку и они 